#for colab

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
!pip uninstall pyglet
!pip install --user pyglet==1.3.2

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0728 05:00:09.591920 140129574807424 abstractdisplay.py:151] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# env = wrap_env(gym.make("MsPacman-v0"))
# show_video()

#main

In [0]:
!pip install chainerrl

     |████████████████████████████████| 112kB 5.0MB/s 
  Created wheel for chainerrl: filename=chainerrl-0.7.0-cp36-none-any.whl size=166687 sha256=3e7ca36a85a0884550776660c3ef5957f778cf4981bcbac82bf53921dde70e8b
  Stored in directory: /root/.cache/pip/wheels/f0/b0/48/4b5e8531a7dcf544a5e238fbc60d7543489c93d7b5e230fa0c
Successfully built chainerrl


In [0]:
from __future__ import print_function
import chainer
import chainer.functions as F
import chainer.links as L
import chainerrl
import numpy as np
import gym
import logging

In [0]:
from tqdm import tqdm

In [0]:
def phi(obs):
    return obs.astype(np.float32)

In [0]:
class A3CLSTMSoftmax(chainer.Chain, chainerrl.agents.a3c.A3CModel, chainerrl.recurrent.RecurrentChainMixin):
    def __init__(self):
        super(A3CLSTMSoftmax, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 16, (11, 9), 1, 0)
            self.conv2 = L.Convolution2D(16, 32, (11, 9), 1, 0)
            self.conv3 = L.Convolution2D(32, 64, (10, 9), 1, 0)
            self.l4p = L.LSTM(14976, 1024) # ポリシーネットワーク
            self.l4v = L.LSTM(14976, 1024) # バリューネットワーク
            self.l5p = L.Linear(1024, 1024)
            self.l5v = L.Linear(1024, 1024)
            self.pi = chainerrl.policies.SoftmaxPolicy(L.Linear(1024, 6))
            self.v = L.Linear(1024, 1)
            
    def pi_and_v(self, state):
        state = np.asarray(state.transpose(0, 3, 1, 2), dtype=np.float32)
        h1 = F.max_pooling_2d(F.relu(self.conv1(state)), ksize=2, stride=2)
        h2 = F.max_pooling_2d(F.relu(self.conv2(h1)), ksize=2, stride=2) 
        h3 = F.max_pooling_2d(F.relu(self.conv3(h2)), ksize=2, stride=2)
        h4p = self.l4p(h3)
        h4v = self.l4v(h3)
        h5p = F.relu(self.l5p(h4p))
        h5v = F.relu(self.l5v(h4v))
        pout = self.pi(h5p) # ポリシーネットワークの出力
        vout = self.v(h5v) # バリューネットワークの出力
        return pout, vout

In [0]:
n_process = 8
outdir = 'result'
chainerrl.misc.set_random_seed(0)
process_seeds = np.arange(n_process)

In [0]:
def make_env(process_idx, test=False):
    env = gym.make('SpaceInvaders-v0')
    process_seed = int(process_seeds[process_idx])
    if not test:
        chainerrl.misc.env_modifiers.make_reward_filtered(env, lambda x: x*0.01)
    if process_idx == 0 and not test:
        env = gym.wrappers.Monitor(env, outdir)
    env_seed = 2 ** 32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    return env

In [0]:
num_episodes = 100000

In [0]:
model = A3CLSTMSoftmax()
optimizer = chainerrl.optimizers.rmsprop_async.RMSpropAsync(lr=0.001, eps=0.1, alpha=0.99)
optimizer.setup(model)

In [0]:
agent = chainerrl.agents.a3c.A3C(
    model, 
    optimizer, 
    t_max = 8,
    gamma = 0.995,
    beta = 0.1,
    phi = phi,
)

In [0]:
gym.logger.set_level(0)
logging.basicConfig(level=logging.DEBUG)

In [0]:
chainerrl.experiments.train_agent_async(
    agent = agent,
    outdir = outdir,
    processes = n_process,
    make_env = make_env,
    profile = True,
    steps = 1000000,
    eval_interval = None,
    max_episode_len = num_episodes,
    logger = gym.logger
)

INFO: Making new env: SpaceInvaders-v0
INFO: Making new env: SpaceInvaders-v0
INFO: Making new env: SpaceInvaders-v0
INFO: Making new env: SpaceInvaders-v0
INFO: Making new env: SpaceInvaders-v0
INFO: Making new env: SpaceInvaders-v0
INFO: Making new env: SpaceInvaders-v0
INFO: Making new env: SpaceInvaders-v0
INFO: Creating monitor directory result
INFO: Starting new video recorder writing to /content/result/openaigym.video.0.210.video000000.mp4
DEBUG: Capturing video frame: path=/content/result/openaigym.video.0.210.video000000.mp4
DEBUG: Starting ffmpeg with "ffmpeg -nostats -loglevel error -y -r 30 -f rawvideo -s:v 160x210 -pix_fmt rgb24 -i - -vcodec libx264 -pix_fmt yuv420p /content/result/openaigym.video.0.210.video000000.mp4"


DEBUG:chainerrl.agents.a3c:t:1 r:0 a:4 pout:SoftmaxDistribution(beta=1.0, min_prob=0.0) logits:[[ 0.10047702 -0.03848258 -0.063296   -0.23922749 -0.17477234 -0.46331877]] probs:[[0.21010141 0.18284352 0.17836238 0.14958815 0.15954737 0.11955714]] entropy:[1.7769206]


DEBUG: Capturing video frame: path=/content/result/openaigym.video.0.210.video000000.mp4


DEBUG:chainerrl.agents.a3c:t:2 r:0.0 a:0 pout:SoftmaxDistribution(beta=1.0, min_prob=0.0) logits:[[ 0.17721948 -0.10529566 -0.08396527 -0.24916878 -0.17999867 -0.5354867 ]] probs:[[0.22899948 0.17263931 0.17636134 0.14950524 0.16021259 0.11228199]] entropy:[1.7698739]


DEBUG: Capturing video frame: path=/content/result/openaigym.video.0.210.video000000.mp4


...

DEBUG:chainerrl.agents.a3c:t:7 r:0.0 a:4 pout:SoftmaxDistribution(beta=1.0, min_prob=0.0) logits:[[ 0.12597479 -0.16489074 -0.08105923 -0.23151934 -0.19576657 -0.5424179 ]] probs:[[0.22234808 0.16623102 0.18076721 0.15551622 0.16117693 0.11396045]] entropy:[1.7729106]


DEBUG: Capturing video frame: path=/content/result/openaigym.video.0.210.video000000.mp4


DEBUG:chainerrl.agents.a3c:t:8 r:0.0 a:1 pout:SoftmaxDistribution(beta=1.0, min_prob=0.0) logits:[[ 0.12220032 -0.17342998 -0.07492602 -0.23524088 -0.19677885 -0.5447308 ]] probs:[[0.22198617 0.16517156 0.18226993 0.1552713  0.16135968 0.11394138]] entropy:[1.772862]


DEBUG: Capturing video frame: path=/content/result/openaigym.video.0.210.video000000.mp4


DEBUG:chainerrl.agents.a3c:pi_loss:[-1.5852811] v_loss:[[0.00061139]]
DEBUG:chainerrl.agents.a3c:grad norm:322.08468898218257
/usr/local/lib/python3.6/dist-packages/chainerrl/misc/async_.py:151: AbnormalExitWarning: Process #0 (pid=210) was terminated by signal 9
  category=AbnormalExitWarning,


In [0]:
#-*- coding: utf-8 -*-
"""
スペースインベーダーゲームの学習（A3Cバージョン）
Copyright(c) Hiromitsu Nishizaki and Koji Makino All Rrights Reserved.
"""
from __future__ import print_function
import chainer
import chainer.functions as F
import chainer.links as L
import chainerrl
import numpy as np
import gym
import logging

# 型変換用関数
def phi(obs):
    return obs.astype(np.float32)

# A3C FeedForward Softmax
class A3CLSTMSoftmax(chainer.Chain, chainerrl.agents.a3c.A3CModel, chainerrl.recurrent.RecurrentChainMixin):
    
    # ポリシーネットワークと評価関数でCNN部分は共通化する
    def __init__(self):
        super(A3CLSTMSoftmax, self).__init__()        
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 16, (11, 9), 1, 0)  # 1層目の畳み込み層（チャンネル数は16）
            self.conv2 = L.Convolution2D(16, 32, (11, 9), 1, 0) # 2層目の畳み込み層（チャンネル数は32）
            self.conv3 = L.Convolution2D(32, 64, (10, 9), 1, 0) # 2層目の畳み込み層（チャンネル数は64）
            self.l4p = L.LSTM(14976, 1024) #ポリシーネットワーク
            self.l4v = L.LSTM(14976, 1024) #バリューネットワーク
            self.l5p = L.Linear(1024, 1024)
            self.l5v = L.Linear(1024, 1024)            
            self.pi = chainerrl.policies.SoftmaxPolicy(L.Linear(1024, 6)) # ポリシーネットワーク
            self.v = L.Linear(1024, 1) # バリューネットワーク

    def pi_and_v(self, state):
        state = np.asarray(state.transpose(0, 3, 1, 2), dtype=np.float32)
        h1 = F.max_pooling_2d(F.relu(self.conv1(state)), ksize=2, stride=2)
        h2 = F.max_pooling_2d(F.relu(self.conv2(h1)), ksize=2, stride=2) 
        h3 = F.max_pooling_2d(F.relu(self.conv3(h2)), ksize=2, stride=2) #ここまでは共通
        h4p = self.l4p(h3)
        h4v = self.l4v(h3)
        h5p = F.relu(self.l5p(h4p)) 
        h5v = F.relu(self.l5p(h4v))
        pout = self.pi(h5p) #ポリシーネットワークの出力
        vout = self.v(h5v) #バリューネットワークの出力
        return pout, vout


# メイン関数
def main():    

    # 初期設定（プロセス数は8）
    n_process = 4
    outdir = 'result'

    # スペースインベーダー環境の設定
    chainerrl.misc.set_random_seed(0)
    process_seeds = np.arange(n_process)

    def make_env(process_idx, test=False):
        env = gym.make('SpaceInvaders-v0')
        process_seed = int(process_seeds[process_idx])
        if not test:
            chainerrl.misc.env_modifiers.make_reward_filtered(env, lambda x: x * 0.01)
        if process_idx == 0 and not test:
            env = gym.wrappers.Monitor(env, outdir, video_callable=(lambda ep: ep % 1 == 0), force=True)
        env_seed = 2 ** 32 - 1 - process_seed if test else process_seed
        env.seed(env_seed)
        return env
    
    
    # 強化学習のパラメータ        
    num_episodes = 10  #試行回数

    # DQNのセットアップ
    model = A3CLSTMSoftmax()
    optimizer = chainerrl.optimizers.rmsprop_async.RMSpropAsync(lr=0.001, eps=0.1, alpha=0.99)
    optimizer.setup(model)
    
    agent = chainerrl.agents.a3c.A3C(
        model, optimizer, t_max=8, gamma=0.995, beta=0.1, 
        phi=phi
    )
    
    # DEBUG用にログを表示
    gym.logger.set_level(0)
    logging.basicConfig(level=logging.DEBUG)

    # エピソードの試行＆強化学習スタート（トレーナーを利用）
    chainerrl.experiments.train_agent_async(
            agent=agent,
            outdir=outdir,
            processes=n_process,
            make_env=make_env,
            profile=True,
            steps=1000000,
            eval_interval=None,
            max_episode_len=num_episodes,
            logger=gym.logger)

if __name__ == '__main__':
    main()